In [ ]:
import os
os.chdir('..')
import torch

# 04 - FGSM Attack

Evaluate the robustness of the LSTM model against a Fast Gradient Sign Method (FGSM) adversarial attack.

In [ ]:
from src.models import LSTMModel
from src.attacks import fgsm_attack
from src.data import prepare_data

model = LSTMModel(1, 64, 1)
model.load_state_dict(torch.load("models/best_model.pth", weights_only=True))
model.eval()

In [ ]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, scaler = prepare_data()
with torch.no_grad():
    predictions = model(X_test_seq)

y_pred = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test_seq.reshape(-1, 1))

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)

X_test_adv = fgsm_attack(model, X_test_seq, y_test_seq, 1) # Epsilon is equal to 1 for the moment, I have to set a relative epsilon value compared to the data scale (AAPL)

with torch.no_grad():
    predictions_adv = model(X_test_adv)
y_pred_adv = scaler.inverse_transform(predictions_adv)

mae_adv = mean_absolute_error(y_test, y_pred_adv)
rmse_adv = root_mean_squared_error(y_test, y_pred_adv)

print(f"Original MAE: {mae:.4f}, RMSE: {rmse:.4f}")
print(f"Adversarial MAE: {mae_adv:.4f}, RMSE: {rmse_adv:.4f}")